In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import optimizers
import itertools
import os
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
fulltrain_df=pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv',header=None)
test_df=pd.read_csv('/kaggle/input/heartbeat/mitbih_test.csv',header=None)

In [ ]:
fulltrain_df = fulltrain_df.sample(frac=1).reset_index(drop=True)

train_df, val_df = train_test_split(fulltrain_df, test_size = 0.33)

In [ ]:
train_df[187]=train_df[187].astype(int)
equilibre=train_df[187].value_counts()
print(equilibre)

In [ ]:
c=train_df.groupby(187,group_keys=False).apply(lambda train_df : train_df.sample(1))
c

In [ ]:
 plt.plot(c.iloc[0,:186])

In [ ]:
plt.plot(c.iloc[1,:186])

In [ ]:
plt.plot(c.iloc[2,:186])

In [ ]:
plt.plot(c.iloc[3,:186])

In [ ]:
plt.plot(c.iloc[4,:186])

In [ ]:
print(train_df[187])

In [ ]:
train_df[187]=train_df[187].astype(int)
equilibre=train_df[187].value_counts()
print(equilibre)

In [ ]:
plt.pie(equilibre, labels=['N','S','V','F','Q'], colors=['red','green','blue','skyblue','orange'],autopct='%1.1f%%',radius=1.5,textprops={'fontsize': 13})
p=plt.gcf()
plt.show()

In [ ]:
from sklearn.utils import resample
df_1=train_df[train_df[187]==1]
df_2=train_df[train_df[187]==2]
df_3=train_df[train_df[187]==3]
df_4=train_df[train_df[187]==4]
df_0=(train_df[train_df[187]==0]).sample(n=48483,random_state=42,replace=True)

df_1_upsample=resample(df_1,replace=True,n_samples=30000,random_state=123)
df_2_upsample=resample(df_2,replace=True,n_samples=30000,random_state=124)
df_3_upsample=resample(df_3,replace=True,n_samples=30000,random_state=125)
df_4_upsample=resample(df_4,replace=True,n_samples=30000,random_state=126)

train_df=pd.concat([df_0,df_1_upsample,df_2_upsample,df_3_upsample,df_4_upsample])

In [ ]:
train_df[187]=train_df[187].astype(int)
equilibre=train_df[187].value_counts()
print(equilibre)

In [ ]:
c=train_df.groupby(187,group_keys=False).apply(lambda train_df : train_df.sample(1))
c

In [ ]:
plt.plot(c.iloc[0,:186] )
plt.plot(c.iloc[1,:186] )
plt.plot(c.iloc[2,:186] )
plt.plot(c.iloc[3,:186] )
plt.plot(c.iloc[4,:186] )

In [ ]:
plt.pie(equilibre, labels=['N','S','V','F','Q'], colors=['red','green','blue','skyblue','orange'],autopct='%1.1f%%',radius=1.5,textprops={'fontsize': 13})
p=plt.gcf()
plt.show()

In [ ]:
target_train=train_df[187]
Y_train=to_categorical(target_train)

target_val=val_df[187]
Y_val=to_categorical(target_val)


X_train=train_df.iloc[:,:186].values
X_val=val_df.iloc[:,:186].values

X_train = X_train.reshape(len(X_train), X_train.shape[1],1)
X_val = X_val.reshape(len(X_val), X_val.shape[1],1)

In [ ]:
target_test=test_df[187]
Y_test=to_categorical(target_test)
X_test=test_df.iloc[:,:186].values
X_test = X_test.reshape(len(X_test), X_test.shape[1],1)

In [ ]:
import keras

from sklearn.preprocessing import MinMaxScaler,StandardScaler
from keras.models import Model, Sequential
from keras.layers import Input,Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint


def get_model():
        im_shape=(X_train.shape[1],1)
        inputs_cnn=Input(shape=(im_shape), name='inputs_cnn')
        

        
        x0 = keras.layers.Conv1D(128,8, activation='relu',input_shape=im_shape)(inputs_cnn)  
        x0= keras.layers.BatchNormalization()(x0)
        pool0=MaxPool1D(pool_size=(4), strides=(2), padding="same")(x0)

        

        x1 = keras.layers.Conv1D(128,8, activation='relu',input_shape=im_shape)(pool0) 
        x1= keras.layers.BatchNormalization()(x1)
        pool1=MaxPool1D(pool_size=(4), strides=(2), padding="same")(x1)

        
        x2 = keras.layers.Conv1D(128,6, activation='relu',input_shape=im_shape)(pool1) 
        x2 = keras.layers.BatchNormalization()(x2)
        pool2=MaxPool1D(pool_size=(3), strides=(2), padding="same")(x2)

        
        x3 = keras.layers.Conv1D(128,6, activation='relu',input_shape=im_shape)(pool2) 
        x3 = keras.layers.BatchNormalization()(x3)
        pool3=MaxPool1D(pool_size=(3), strides=(2), padding="same")(x3)

        
        x4 = keras.layers.Conv1D(128,4, activation='relu',input_shape=im_shape)(pool3) 
        x4 = keras.layers.BatchNormalization()(x4)
        pool4=MaxPool1D(pool_size=(3), strides=(1), padding="same")(x4)

        
        x5 = keras.layers.Conv1D(128,2, activation='relu',input_shape=im_shape)(pool4) 
        x5 = keras.layers.BatchNormalization()(x5)
        pool5=MaxPool1D(pool_size=(3), strides=(1), padding="same")(x5)

        
        x6 = keras.layers.Conv1D(128,2, activation='relu',input_shape=im_shape)(pool5) 
        x6 = keras.layers.BatchNormalization()(x6)
        pool6=MaxPool1D(pool_size=(2), strides=(1), padding="same")(x6)

        
        x7 = keras.layers.Conv1D(128,2, activation='relu',input_shape=im_shape)(pool6) 
        x7 = keras.layers.BatchNormalization()(x7)
        pool7=MaxPool1D(pool_size=(2), strides=(1), padding="same")(x7)
    
    
        x = Flatten()(pool7)
        x = keras.layers.BatchNormalization()(x)
        
        dense_1 = Dense(32, activation='relu')(x)
        dense_2 = Dense(32, activation='relu')(dense_1)

        out = keras.layers.Dense(5, activation='softmax')(dense_2)
        return keras.Model(inputs=inputs_cnn, outputs=out)

In [ ]:
model = get_model()



model.compile(optimizer = 'adam' ,loss= "categorical_crossentropy", metrics=["accuracy"])
history=model.fit(X_train, Y_train,epochs=10, batch_size=32,validation_data=(X_val,Y_val))

In [ ]:
model.summary()

In [ ]:
score, acc = model.evaluate(X_test, Y_test,
                            batch_size=50)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
train_loss=history.history['loss']
val_loss=history.history['val_loss']
train_acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
xc=range(10)

In [ ]:
plt.figure(1,figsize=(10,8),facecolor='skyblue')
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('Number of Epochs',fontsize=18)
plt.ylabel('Loss',fontsize=18)
plt.title('Training_loss vs Validation_loss',fontsize=20, fontweight='bold')
plt.grid(True)
plt.legend(['Training','Validation'])
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])


In [ ]:
plt.figure(1,figsize=(10,8),facecolor='skyblue')
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('Number of Epochs',fontsize=18)
plt.ylabel('Accuracy',fontsize=18)
plt.title('Training_accuracy vs Validation_accuracy',fontsize=20, fontweight='bold')
plt.grid(True)
plt.legend(['Training','Validation'])
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

In [ ]:

import seaborn as sns
from sklearn.metrics import confusion_matrix,classification_report

y_pred= model.predict(X_test)
#print(X_test)
y_pred=np.argmax(y_pred,axis=1)
#print(y_pred)

#y_pred=model.predict_classes(X_test)


#target_names= classes['N', 'S', 'V', 'F', 'Q']
#print(classification_report(np.argmax(y_test,axis=1),y_pred,target_names=))
sns.set(font_scale=1.5)
cm=confusion_matrix(np.argmax(Y_test,axis=1),y_pred)
print(cm)

import itertools
from sklearn.metrics import plot_confusion_matrix

    
#plt.figure(figsize=(10, 10))
#plot_confusion_matrix(cm, classes=['N', 'S', 'V', 'F', 'Q'],normalize=True,
                     # title='Confusion matrix, with normalization')

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns


import numpy as np
rounded_labels=np.argmax(Y_test, axis=1)
rounded_labels[1]

cm = confusion_matrix(rounded_labels, y_pred)
# Normalise
cmn = cm.astype('float') /cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels= ['N', 'S', 'V', 'F', 'Q'], yticklabels=['N', 'S', 'V', 'F', 'Q'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show(block=False)
